

### 3. Preprocessing
- **Purpose**: Ensures that your data is clean and ready for analysis.
- **Content**:
   - Handle missing values, duplicates, and outliers.
   - Convert data types for consistency (e.g., date-time conversion).
   - Create any necessary derived columns (e.g., offer validity period calculations).
   - Merge datasets (e.g., combining transcript and profile data) for comprehensive analysis.
   - Implement your identified metrics related to offer funnel, spending lift, and demographic responses.
   - **Encode Features**: Use one-hot encoding for categorical variables like `offer_type` and `gender`, and scale or bin continuous variables like income or age as needed.
   - Save the processed data in the right folder
  
   
---

In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('../data/raw/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/raw/profile.json', orient='records', lines=True)
transcript = pd.read_json('../data/raw/transcript.json', orient='records', lines=True)

In [2]:
print(portfolio.info())
print(portfolio.describe())
portfolio.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes
None
          reward  difficulty   duration
count  10.000000   10.000000  10.000000
mean    4.200000    7.700000   6.500000
std     3.583915    5.831905   2.321398
min     0.000000    0.000000   3.000000
25%     2.000000    5.000000   5.000000
50%     4.000000    8.500000   7.000000
75%     5.000000   10.000000   7.000000
max    10.000000   20.000000  10.000000


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [3]:
print(profile.info())
print(profile.describe())
profile.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB
None
                age  became_member_on         income
count  17000.000000      1.700000e+04   14825.000000
mean      62.531412      2.016703e+07   65404.991568
std       26.738580      1.167750e+04   21598.299410
min       18.000000      2.013073e+07   30000.000000
25%       45.000000      2.016053e+07   49000.000000
50%       58.000000      2.017080e+07   64000.000000
75%       73.000000      2.017123e+07   80000.000000
max      118.000000      2.018073e+07  120000.000000


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [4]:
print(transcript.info())
print(transcript.describe())
transcript.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB
None
                time
count  306534.000000
mean      366.382940
std       200.326314
min         0.000000
25%       186.000000
50%       408.000000
75%       528.000000
max       714.000000


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [5]:
# 1. Handle Missing Values, Duplicates, and Outliers
# Check for missing values
print(portfolio.isnull().sum())
print(profile.isnull().sum())
print(transcript.isnull().sum())

reward        0
channels      0
difficulty    0
duration      0
offer_type    0
id            0
dtype: int64
gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64
person    0
event     0
value     0
time      0
dtype: int64


### 1. portfolio

In [6]:
# First, apply the channels encoding
channels_encoded = portfolio['channels'].str.join('|').str.get_dummies()
portfolio = pd.concat([portfolio, channels_encoded], axis=1)

# Drop the original 'channels' column since it's now encoded
portfolio.drop(columns=['channels'], inplace=True)

# Now, perform one-hot encoding for 'offer_type'
portfolio = pd.get_dummies(portfolio, columns=['offer_type'], prefix='offer_type',dtype=int)

# Display the updated DataFrame
portfolio.head()


,reward,difficulty,duration,id,email,mobile,social,web,offer_type_bogo,offer_type_discount,offer_type_informational
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,1,0,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0
2,0,0,4,3f207df678b143eea3cee63160fa8bed,1,1,0,1,0,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,1,0,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,0,1,0


We could drop either offer_type_bogo, offer_type_discount or offer_type_informational since when the other two are 0 it automatically means that it has to be the third offer type. That would reduce redundancy 

In [7]:
# Drop one of the `offer_type` columns ("offer_type_informational")
portfolio.drop(columns=['offer_type_informational'], inplace=True)

# Display the updated DataFrame
portfolio.head()

,reward,difficulty,duration,id,email,mobile,social,web,offer_type_bogo,offer_type_discount
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,1,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0
2,0,0,4,3f207df678b143eea3cee63160fa8bed,1,1,0,1,0,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,0,1


### 2. profile

In [8]:
profile.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [9]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


When we inspect the profile dataset we can see that most of the time the rows where the gender and the income is missing there is also an age of 118 which is highly douptful. so lets write a function that drops all rows, where the gender and the income is NaN and the age is 118. 

In [10]:
#filters out rows where age is 118, gender is NaN, and income is NaN using boolean indexing.
profile = profile[~((profile['age'] == 118) & (profile['gender'].isna()) & (profile['income'].isna()))]

In [11]:
profile.head()

,gender,age,id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0


In [12]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14825 entries, 1 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               14825 non-null  int64  
 2   id                14825 non-null  object 
 3   became_member_on  14825 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 694.9+ KB


In [13]:
# Convert became_member_on to datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

profile.head()

,gender,age,id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0


In [14]:
# Apply one-hot encoding to the 'gender' column
gender_encoded = pd.get_dummies(profile['gender'], prefix='gender', dtype=int)

# Concatenate the encoded columns with the original dataframe and drop the original 'gender' column
profile = pd.concat([profile.drop('gender', axis=1), gender_encoded], axis=1)

profile.head()

,age,id,became_member_on,income,gender_F,gender_M,gender_O
1,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1,0,0
3,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1,0,0
5,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,0,1,0
8,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,0,1,0
12,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,0,1,0


now again we can drop the gender_O column since when the other gender columns are 0 that means that the third option is true 

In [15]:
# Drop one of the `offer_type` columns ("offer_type_informational")
profile.drop(columns=['gender_O'], inplace=True)

# Display the updated DataFrame
profile.head()

,age,id,became_member_on,income,gender_F,gender_M
1,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1,0
3,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1,0
5,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,0,1
8,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,0,1
12,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,0,1


In [16]:
profile['income'].min()

30000.0

for scaling and readability purpose lats convert the income in income/k 

In [17]:
# Divide the income column by 1000 to convert it to thousands
profile['income'] = profile['income'] / 1000

# Rename the column to 'income/k'
profile.rename(columns={'income': 'income/k'}, inplace=True)

profile.head()

,age,id,became_member_on,income/k,gender_F,gender_M
1,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112.0,1,0
3,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100.0,1,0
5,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70.0,0,1
8,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53.0,0,1
12,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51.0,0,1


### 3. transcript

In [18]:
# Normalize the 'value' column, which contains dictionaries
value_columns = pd.json_normalize(transcript['value'])

# Concatenate the new columns to the original transcript DataFrame
transcript = pd.concat([transcript, value_columns], axis=1)

# Drop the original 'value' column
transcript.drop(columns=['value'], inplace=True)

# Check the result
transcript.head()

,person,event,time,offer id,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN


there are some entrys with offer_id and some with offer id lets check if there are rows where these both have a value

In [19]:
# Check for rows where both 'offer id' and 'offer_id' are not NaN
rows_with_both = transcript[transcript['offer id'].notna() & transcript['offer_id'].notna()]

# Display the rows
print(rows_with_both)

Empty DataFrame
Columns: [person, event, time, offer id, amount, offer_id, reward]
Index: []


as we can see they never have a value at the same time so lets merge them together.

In [20]:
# Merge 'offer id' and 'offer_id' into a single column 'offer_id'
transcript['offer_id'] = transcript['offer id'].fillna(transcript['offer_id'])

# Drop the original 'offer id' column if you no longer need it
transcript.drop(columns=['offer id'], inplace=True)

transcript.head()

,person,event,time,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


In [21]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   person    306534 non-null  object 
 1   event     306534 non-null  object 
 2   time      306534 non-null  int64  
 3   amount    138953 non-null  float64
 4   offer_id  167581 non-null  object 
 5   reward    33579 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 14.0+ MB


In [22]:
""" # Fill NaN values in 'amount' and 'reward' columns with 0
transcript['amount']=transcript['amount'].fillna(0)
transcript['reward']=transcript['reward'].fillna(0)

# Fill Nan values in 'offer_id' with '' empty string.
transcript['offer_id']=transcript['offer_id'].fillna('')

transcript.info() """

" # Fill NaN values in 'amount' and 'reward' columns with 0\ntranscript['amount']=transcript['amount'].fillna(0)\ntranscript['reward']=transcript['reward'].fillna(0)\n\n# Fill Nan values in 'offer_id' with '' empty string.\ntranscript['offer_id']=transcript['offer_id'].fillna('')\n\ntranscript.info() "

In [23]:
# One-hot encode the 'event' column
event_encoded = pd.get_dummies(transcript['event'], prefix='event', dtype=int)

# Concatenate the new columns to the original DataFrame
transcript = pd.concat([transcript, event_encoded], axis=1)

# Drop the original 'event' column
transcript.drop(columns=['event'], inplace=True)

transcript.head()


,person,time,amount,offer_id,reward,event_offer completed,event_offer received,event_offer viewed,event_transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,0,1,0,0


we can drop event_transacrtion, because we know when every other event column is 0 it hast to be transaction

In [24]:
# Drop the 'event_transaction' column since it can be inferred
transcript.drop(columns=['event_transaction'], inplace=True)

transcript.head()

,person,time,amount,offer_id,reward,event_offer completed,event_offer received,event_offer viewed
0,78afa995795e4d85b5d9ceeca43f5fef,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,0,1,0
1,a03223e636434f42ac4c3df47e8bac43,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0,1,0
2,e2127556f4f64592b11af22de27a7932,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,0,1,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,0,1,0
4,68617ca6246f4fbc85e91a2a49552598,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,0,1,0


Our question is who is responsive to each type of offer and how we should present it. that means it is not necessary to keep amount and reward. 

In [25]:
# Drop the 'amount' and 'reward' columns
transcript.drop(columns=['amount', 'reward'], inplace=True)

# Display the DataFrame to verify the changes
transcript.head()

,person,time,offer_id,event_offer completed,event_offer received,event_offer viewed
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,0,1,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,0,1,0
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0,1,0


It is time now to merge all these 3 dataframes together. 

In [26]:
# First, merge profile and transcript on the user IDs (person in transcript, id in profile)
merged_data = transcript.merge(profile, left_on='person', right_on='id', how='left')

# Drop the redundant 'id' column from profile as it's already represented by 'person'
merged_data.drop(columns=['id'], inplace=True)

# Now, merge the portfolio with the merged data based on the offer_id (offer_id in merged_data, id in portfolio)
final_merged_data = merged_data.merge(portfolio, left_on='offer_id', right_on='id', how='left')

# Drop the redundant 'id' column from portfolio
final_merged_data.drop(columns=['id'], inplace=True)

# Display the final merged DataFrame
final_merged_data.head()

,person,time,offer_id,event_offer completed,event_offer received,event_offer viewed,age,became_member_on,income/k,gender_F,gender_M,reward,difficulty,duration,email,mobile,social,web,offer_type_bogo,offer_type_discount
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0,75.0,2017-05-09,100.0,1.0,0.0,5.0,5.0,7.0,1.0,1.0,0.0,1.0,1.0,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,NaN,NaT,NaN,NaN,NaN,5.0,20.0,10.0,1.0,0.0,0.0,1.0,0.0,1.0
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,0,1,0,68.0,2018-04-26,70.0,0.0,1.0,2.0,10.0,7.0,1.0,1.0,0.0,1.0,0.0,1.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,NaN,NaT,NaN,NaN,NaN,2.0,10.0,10.0,1.0,1.0,1.0,1.0,0.0,1.0
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0,1,0,NaN,NaT,NaN,NaN,NaN,10.0,10.0,5.0,1.0,1.0,1.0,1.0,1.0,0.0


In [27]:
final_merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   person                 306534 non-null  object        
 1   time                   306534 non-null  int64         
 2   offer_id               167581 non-null  object        
 3   event_offer completed  306534 non-null  int32         
 4   event_offer received   306534 non-null  int32         
 5   event_offer viewed     306534 non-null  int32         
 6   age                    272762 non-null  float64       
 7   became_member_on       272762 non-null  datetime64[ns]
 8   income/k               272762 non-null  float64       
 9   gender_F               272762 non-null  float64       
 10  gender_M               272762 non-null  float64       
 11  reward                 167581 non-null  float64       
 12  difficulty             167581 non-null  floa

In [28]:
# Fill missing numeric values with 0

final_merged_data['reward']= final_merged_data['reward'].fillna(0)

# Fill missing offer_id values with 'No Offer'
final_merged_data['offer_id']= final_merged_data['offer_id'].fillna('No Offer')

#if there is no demographic data we can drop these row since it is important to identify the usergroup for the offer.
final_merged_data.dropna(subset=['age', 'became_member_on', 'income/k', 'gender_F', 'gender_M'], inplace=True)

#we also need to know which type of offer each participant got so lets drop the NaN values for offer_type_bogo and offer_type_discount.
final_merged_data.dropna(subset=['offer_type_discount','offer_type_bogo'], inplace=True)

# Reset the index of the final_merged_data DataFrame
final_merged_data.reset_index(drop=True, inplace=True)


This is the whole dataframe fitted in one big dataframe now we can expand that with our new metrics. 

In [29]:
final_merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148805 entries, 0 to 148804
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   person                 148805 non-null  object        
 1   time                   148805 non-null  int64         
 2   offer_id               148805 non-null  object        
 3   event_offer completed  148805 non-null  int32         
 4   event_offer received   148805 non-null  int32         
 5   event_offer viewed     148805 non-null  int32         
 6   age                    148805 non-null  float64       
 7   became_member_on       148805 non-null  datetime64[ns]
 8   income/k               148805 non-null  float64       
 9   gender_F               148805 non-null  float64       
 10  gender_M               148805 non-null  float64       
 11  reward                 148805 non-null  float64       
 12  difficulty             148805 non-null  floa

In [30]:
final_merged_data.head()

,person,time,offer_id,event_offer completed,event_offer received,event_offer viewed,age,became_member_on,income/k,gender_F,gender_M,reward,difficulty,duration,email,mobile,social,web,offer_type_bogo,offer_type_discount
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0,75.0,2017-05-09,100.0,1.0,0.0,5.0,5.0,7.0,1.0,1.0,0.0,1.0,1.0,0.0
1,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,0,1,0,68.0,2018-04-26,70.0,0.0,1.0,2.0,10.0,7.0,1.0,1.0,0.0,1.0,0.0,1.0
2,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d,0,1,0,65.0,2018-02-09,53.0,0.0,1.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,0.0
3,2eeac8d8feae4a8cad5a6af0499a211d,0,3f207df678b143eea3cee63160fa8bed,0,1,0,58.0,2017-11-11,51.0,0.0,1.0,0.0,0.0,4.0,1.0,1.0,0.0,1.0,0.0,0.0
4,aa4862eba776480b8bb9c68455b8c2e1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,61.0,2017-09-11,57.0,1.0,0.0,5.0,20.0,10.0,1.0,0.0,0.0,1.0,0.0,1.0


with this dataframe it should be possible to tackle the next step which is EDA. Let's save it

In [31]:
# Save the final merged data as a CSV file
final_merged_data.to_csv('../data/processed/merged_data.csv', index=False)